<a href="https://colab.research.google.com/github/JeongHunShin91/pytorch_training/blob/main/RNN_%EC%9E%84%EB%B2%A0%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

In [3]:
vocab = list(set(sentence))
print(vocab)

['medicine', 'best', 'the', 'is', 'Repeat', 'for', 'memory']


In [4]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0

In [5]:
print(word2index)

{'medicine': 1, 'best': 2, 'the': 3, 'is': 4, 'Repeat': 5, 'for': 6, 'memory': 7, '<unk>': 0}


In [6]:
print(word2index['memory'])

7


In [7]:
# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'medicine', 2: 'best', 3: 'the', 4: 'is', 5: 'Repeat', 6: 'for', 7: 'memory', 0: '<unk>'}


In [8]:
print(index2word[2])

best


In [10]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [11]:
X, Y = build_data(sentence, word2index)

In [12]:
print(X)
print(Y)

tensor([[5, 4, 3, 2, 1, 6]])
tensor([[4, 3, 2, 1, 6, 7]])


In [14]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [53]:
# 모델 생성
model = Net(8, 1, 1, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [54]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)

tensor([[ 0.3802, -0.4223,  0.8038, -0.0536, -0.0152,  0.4917, -0.8380, -0.1885],
        [ 0.3622, -0.3293,  0.6784,  0.1016,  0.0493,  0.4388, -0.6847, -0.4067],
        [ 0.3864, -0.4543,  0.8470, -0.1071, -0.0374,  0.5100, -0.8907, -0.1133],
        [ 0.3595, -0.3156,  0.6599,  0.1245,  0.0587,  0.4310, -0.6622, -0.4388],
        [ 0.3901, -0.4734,  0.8728, -0.1390, -0.0506,  0.5209, -0.9222, -0.0684],
        [ 0.3927, -0.4866,  0.8907, -0.1611, -0.0598,  0.5284, -0.9440, -0.0373]],
       grad_fn=<ViewBackward0>)


In [46]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [55]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.2751 
Repeat best best best best best best

[41/201] 2.2096 
Repeat best best best best best best

[81/201] 2.1458 
Repeat best best best best best best

[121/201] 2.0831 
Repeat best the best the best best

[161/201] 2.0206 
Repeat best the best the best best

[201/201] 1.9584 
Repeat the the best the best best

